**Author :  Ong Cheng Kei TP055620** <br>
**Description :**
<br>This file contains code to build the serialize Recipes5k, Nutrition5k, Food101 into tfrecord files for better efficiency in model training.<br>
This file is also a prerequisite for running the *build_model.ipynb* as it creates the one hot encoder needed to encode the categorical data<br> 
The output will be a set of tfrecord files for each dataset stored in *../Food Datasets/final-dataset/tfrecord*

# Setup

In [1]:
import json
from pathlib import Path
from types import SimpleNamespace

import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

c:\Users\kenan\anaconda3\envs\food_v5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Build a universal one hot encoder that encodes cross-dataset category and ingredients 

In [3]:
# create a universal one-hot encoder that encodes categories and ingredients across datasets
class OneHotEncoder:
    # Initializes the encoder with lists of all categories and ingredients.
    # Sorts the lists of categories and ingredients.
    # Calls private methods to encode these sorted lists into integer mappings.
    def __init__(self, all_category_list, all_ingredient_list):
        self.all_food_categories = all_category_list
        self.all_food_categories.sort()
        self.all_food_categories_integer_encoded = (
            self.__encode_categories_to_integers()
        )
        self.all_ingredients = all_ingredient_list
        self.all_ingredients.sort()
        self.all_ingredients_integer_encoded = self.__encode_ingredients_to_integers()

    # Takes a category name as input.
    # Retrieves the integer index for the category and asserts its existence.
    # Converts the integer index to a one-hot encoded vector using keras.utils.to_categorical.
    # Returns the one-hot encoded vector.
    def get_category_one_hot_encoding(self, category_name):
        index = self.all_food_categories_integer_encoded[category_name]
        assert index is not None, f"{category_name} does not have an integer mapping"
        num_classes = len(self.all_food_categories)
        return keras.utils.to_categorical(index, num_classes, dtype="uint8")

    # Takes a list of ingredient names.
    # Maps each ingredient name to its integer index.
    # Uses tf.keras.layers.CategoryEncoding to create a multi-hot encoded vector for the ingredients.
    # Returns the multi-hot encoded vector cast to tf.uint8.
    def get_ingredients_one_hot_encoding(self, ingredient_list):
        ingredient_list = list(
            map(lambda x: self.__transform_ingredient_to_integer(x), ingredient_list)
        )
        multi_one_hot_layer = tf.keras.layers.CategoryEncoding(
            num_tokens=len(self.all_ingredients), output_mode="multi_hot"
        )
        return tf.cast(multi_one_hot_layer(ingredient_list), dtype=tf.uint8)

    # Transforms an ingredient name to its integer index.
    # Asserts the existence of the mapping.
    def __transform_ingredient_to_integer(self, ingredient_name):
        index = self.all_ingredients_integer_encoded[ingredient_name]
        assert index is not None, f"{ingredient_name} does not have an integer mapping"
        return index

    # Creates a dictionary mapping each category name to a unique integer index.
    # Uses enumerate to generate the indices.
    def __encode_categories_to_integers(self):
        return {
            category_name: index
            for index, category_name in enumerate(self.all_food_categories)
        }

    # Similar to __encode_categories_to_integers, it creates a dictionary mapping each ingredient name to a unique integer index.
    def __encode_ingredients_to_integers(self):
        return {
            ingredient_name: index
            for index, ingredient_name in enumerate(self.all_ingredients)
        }

The code initializes an OneHotEncoder class to handle one-hot and multi-hot encoding for categories and ingredients across different datasets (Recipes5k, Nutrition5k, Food101). This encoder is essential for preparing the data before converting it to TFRecord files, which is the next step in the pipeline for efficient model training.

# Build dataset loaders for each dataset

In [4]:
# used to handle the loading and processing of images and metadata for different datasets.
class DatasetLoader:
    # Initializes the loader with image and metadata directory paths, and the dataset name
    # Loads the metadata file.
    # Extracts all categories and ingredients from the metadata.
    def __init__(self, image_dir, metadata_dir, dataset_name):
        self.image_dir = Path(image_dir)
        self.metadata_dir = Path(metadata_dir)
        self.name = dataset_name
        self.metadata = self.load_metadata(
            self.metadata_dir / ("{dataset}_metadata.csv".format(dataset=dataset_name))
        )
        # Default : all_files = metadata, since metadata consists of records of all files
        self.all_files = self.metadata.copy()
        self.all_categories = self.extract_all_categories()
        self.all_ingredients = self.extract_all_ingredients()

    # oads an image from a given path and converts it to a tensor of shape (224, 224).
    # Returns the image tensor cast to tf.uint8.
    def load_image_to_arr(self, path):
        image = tf.keras.preprocessing.image.load_img(path)
        img_tensor = tf.keras.preprocessing.image.img_to_array(image, dtype="uint8")
        img_tensor = tf.image.resize(img_tensor, (224, 224))
        return tf.cast(img_tensor, tf.uint8)

    # Reads the metadata file from the given path.
    # Adds a dataset_name column to the metadata.
    # Returns the modified metadata.
    def load_metadata(self, path):
        metadata = pd.read_csv(path, sep="\t")
        new_metadata = metadata.copy()
        new_metadata["dataset_name"] = self.name
        return new_metadata

    # Extracts unique categories from the metadata.
    # Returns a list of unique categories.
    def extract_all_categories(self):
        return self.metadata["Category"].unique().tolist()

    # Extracts unique ingredients from the metadata by iterating through the ingredient lists.
    # Returns a list of unique ingredients.
    def extract_all_ingredients(self):
        unique_ingredients = set()
        for ingredient_list in self.metadata["Ingredients"]:
            ingredient_list = ingredient_list.split(",")
            unique_ingredients.update(ingredient_list)
        return [*unique_ingredients]

    # Extracts pointers to metadata and dataset names.
    # Returns a DataFrame with metadata indices and dataset names.
    def extract_file_pointers(self):
        dataset_name_col = self.all_files["dataset_name"]
        index_col = self.all_files.index
        return pd.DataFrame(
            {"metadata_index": index_col, "dataset_name": dataset_name_col}
        )

    # Loads an image and its metadata as tensors.
    # Encodes the image to JPEG or PNG format.
    # Extracts nutritional values and category information.
    # Returns the image tensor and a dictionary of metadata tensors.
    def get_tensors(self, index):
        img_dir = self.image_dir
        row = self.all_files.loc[index]
        img_path = img_dir / row["Category"] / row["ID/File Name"]
        img_tensor = self.load_image_to_arr(img_path)
        if img_path.suffix == ".jpeg" or img_path.suffix == ".jpg":
            img_tensor = tf.io.encode_jpeg(img_tensor, format="rgb")
        elif img_path.suffix == ".png":
            img_tensor = tf.io.encode_png(img_tensor)
        else:
            assert False, "Invalid image format present"
        calorie_tensor = row["Calorie(kcal)"]
        carbs_tensor = row["Carbohydrate(g)"]
        protein_tensor = row["Protein(g)"]
        fat_tensor = row["Fat(g)"]
        return img_tensor, {
            "category_output": tf.constant(row["Category"]),
            "calorie_output": tf.constant(calorie_tensor),
            "carbs_output": tf.constant(carbs_tensor),
            "protein_output": tf.constant(protein_tensor),
            "fat_output": tf.constant(fat_tensor),
            "ingredients_output": tf.constant(row["Ingredients"]),
        }

    # Flattens the image and metadata tensors into a list of numpy arrays.
    # Returns the flattened list.
    def flatten_tensors(self, tensor):
        result = []
        img_data = tensor[0].numpy()
        others_data = [value.numpy() for key, value in tensor[1].items()]
        result.append(img_data)
        result.extend(others_data)
        return result

    # Returns the length of the metadata.
    def __len__(self):
        return len(self.metadata)

In [5]:
# A subclass of DatasetLoader for handling the Recipes5k dataset.
class Recipes5k(DatasetLoader):
    # Initializes the loader for Recipes5k by calling the parent constructor.
    def __init__(self, image_dir, metadata_dir):
        super().__init__(image_dir, metadata_dir, "recipes5k")

In [6]:
# A subclass of DatasetLoader for handling the Nutrition5k dataset.
class Nutrition5k(DatasetLoader):
    # Initializes the loader for Nutrition5k by calling the parent constructor.
    # Modifies all_files to read from a specific file that contains image-level metadata.
    def __init__(self, image_dir, metadata_dir):
        super().__init__(image_dir, metadata_dir, "nutrition5k")
        # Modify all_files since nutrition5k metadata only consists dish_level metadata not image_level
        self.all_files = pd.read_csv(self.metadata_dir / "nutrition5k_all_images.csv")

    # Override method from DatasetLoader
    # Extracts dish-level metadata for nutritional values.
    def get_tensors(self, index):
        img_dir = self.image_dir
        row = self.all_files.loc[index]
        img_path = img_dir / "generic" / row["dish_id"] / row["ID/File Name"]
        img_tensor = self.load_image_to_arr(img_path)
        if img_path.suffix == ".jpeg" or img_path.suffix == ".jpg":
            img_tensor = tf.io.encode_jpeg(img_tensor, format="rgb")
        elif img_path.suffix == ".png":
            img_tensor = tf.io.encode_png(img_tensor)
        else:
            assert False, "Invalid image format present"
        dish_metadata_row = self.metadata.loc[
            self.metadata["dish_id"] == row["dish_id"]
        ].squeeze()
        calorie_tensor = dish_metadata_row["Calorie(kcal)"]
        carbs_tensor = dish_metadata_row["Carbohydrate(g)"]
        protein_tensor = dish_metadata_row["Protein(g)"]
        fat_tensor = dish_metadata_row["Fat(g)"]
        return img_tensor, {
            "category_output": tf.constant(dish_metadata_row["Category"]),
            "calorie_output": tf.constant(calorie_tensor),
            "carbs_output": tf.constant(carbs_tensor),
            "protein_output": tf.constant(protein_tensor),
            "fat_output": tf.constant(fat_tensor),
            "ingredients_output": tf.constant(dish_metadata_row["Ingredients"]),
        }

    # Overrding the method from DatasetLoader
    def __len__(self):
        return len(self.all_files)

In [7]:
# A subclass of DatasetLoader for handling the Food101 dataset.
class Food101(DatasetLoader):
    # Initializes the loader for Food101 by calling the parent constructor.
    def __init__(self, image_dir, metadata_dir):
        super().__init__(image_dir, metadata_dir, "food101")

# Initializing one hot encoder

In [8]:
# Get all the categories and ingredients from all datasets

# Initialize dataset loader without one-hot encoder to get all unique category and ingredients from each dataset
RECIPES5K = Recipes5k(
    image_dir="D:\\food_v5\\Food Datasets\\final-dataset\\images",
    metadata_dir="D:\\food_v5\\Food Datasets\\final-dataset\\metadata",
)
NUTRITION5K = Nutrition5k(
    image_dir="D:\\food_v5\\Food Datasets\\final-dataset\\images",
    metadata_dir="D:\\food_v5\\Food Datasets\\final-dataset\\metadata",
)
FOOD101 = Food101(
    image_dir="D:\\food_v5\\Food Datasets\\final-dataset\\images",
    metadata_dir="D:\\food_v5\\Food Datasets\\final-dataset\\metadata",
)

DATASETS = [RECIPES5K, NUTRITION5K, FOOD101]
DATASETS_NAME = [x.name for x in DATASETS]


def create_one_hot_encoder(datasets):
    all_categories = []
    all_ingredients = []
    for x in datasets:
        all_categories.extend(x.all_categories)
        all_ingredients.extend(x.all_ingredients)
    all_categories = set(all_categories)
    all_ingredients = set(all_ingredients)
    return OneHotEncoder([*all_categories], [*all_ingredients])

In [9]:
ONE_HOT_ENCODER = create_one_hot_encoder(DATASETS)

defines a base class DatasetLoader and its subclasses to handle different datasets. It initializes dataset loaders for Recipes5k, Nutrition5k, and Food101, extracts unique categories and ingredients, and creates a universal one-hot encoder for encoding these categories and ingredients. This setup is essential for efficiently loading, processing, and preparing the datasets for model training in TensorFlow.

# Building data pipeline that streams the file index and dataset index

In [10]:
# Retrieves and flattens the tensor data for a given file index and dataset index.
def get_file_data(index, dataset_index):
    target_dataset = DATASETS[dataset_index]
    return target_dataset.flatten_tensors(target_dataset.get_tensors(index))

# Builds a data pipeline that streams file and dataset indices.
def build_data_pipeline(datasets, sample_size=None):
    if sample_size is None:
        sample_size = [1.0] * len(datasets)
    assert len(sample_size) == len(
        datasets
    ), "Illegal array of sample sizes provided. Number of sample size does not match number of datasets"
    file_pointers = [
        x.extract_file_pointers().sample(frac=s, random_state=999)
        for x, s in zip(datasets, sample_size)
    ]
    all_file_pointers = pd.concat(file_pointers).sample(frac=1, random_state=999)
    print(f"Total samples : {len(all_file_pointers)}")

    all_file_pointers["dataset_name"] = all_file_pointers["dataset_name"].apply(
        lambda x: DATASETS_NAME.index(x)
    )

    final_dataset = tf.data.Dataset.from_tensor_slices(
        (
            all_file_pointers["metadata_index"].tolist(),
            all_file_pointers["dataset_name"].tolist(),
        )
    )
    return final_dataset

In [11]:
recipes5k_dataset = build_data_pipeline([RECIPES5K])
food101_dataset = build_data_pipeline([FOOD101])
nutrition5k_dataset = build_data_pipeline([NUTRITION5K])

Total samples : 4826
Total samples : 101000
Total samples : 271829


In [12]:
list(recipes5k_dataset.take(2))

[(<tf.Tensor: shape=(), dtype=int32, numpy=2344>,
  <tf.Tensor: shape=(), dtype=int32, numpy=0>),
 (<tf.Tensor: shape=(), dtype=int32, numpy=4556>,
  <tf.Tensor: shape=(), dtype=int32, numpy=0>)]

## Serializing Data Pipeline to TFRecord with TFDS Features

In [13]:
# Defines the structure and types of features in the datasets.
FEATURE_DICTIONARY = tfds.features.FeaturesDict(
    {
        "image_raw": tfds.features.Image(
            shape=(224, 224, 3), doc="Raw bytes of food images encoded with tf.io"
        ),
        "category": tfds.features.Scalar(dtype=tf.string, doc="Category label"),
        "calorie": tfds.features.Scalar(
            dtype=tf.float32, doc="Calorie of the food per gram"
        ),
        "carbs": tfds.features.Scalar(
            dtype=tf.float32, doc="Carbs of the food per gram"
        ),
        "protein": tfds.features.Scalar(
            dtype=tf.float32, doc="Protein of the food per gram"
        ),
        "fat": tfds.features.Scalar(dtype=tf.float32, doc="Fat of the food per gram"),
        "ingredients": tfds.features.Scalar(
            dtype=tf.string, doc="Ingredients of food separated with comma"
        ),
    }
)

### Shard and write to TFRecord file
shreds the dataset and writes it to TFRecord files for better efficiency in model training. It also exports the encoded categories and ingredients

In [14]:
# Splits the dataset into shards and writes them to TFRecord files.
def shard_and_write(dataset, num_shards, path, dataset_name):
    path = Path(path)
    if not path.exists():
        path.mkdir()

    sharded_template_generator = tfds.core.ShardedFileTemplate(
        data_dir=path.as_posix(),
        template="{DATASET}-{SPLIT}-{SHARD_X_OF_Y}.{FILEFORMAT}",
        dataset_name=dataset_name,
        filetype_suffix="tfrecord",
        split="train",
    )
    shard_length = []
    sharded_filepaths = sharded_template_generator.sharded_filepaths(num_shards)
    for i in range(num_shards):
        current_shard = dataset.shard(num_shards, i)
        with tf.io.TFRecordWriter(sharded_filepaths[i].as_posix()) as writer:
            length = 0
            for record in current_shard.as_numpy_iterator():
                data = get_file_data(record[0], record[1])
                example = {
                    "image_raw": data[0],
                    "category": data[1],
                    "calorie": data[2],
                    "carbs": data[3],
                    "protein": data[4],
                    "fat": data[5],
                    "ingredients": data[6],
                }
                example_bytes = FEATURE_DICTIONARY.serialize_example(example)
                writer.write(example_bytes)
                length += 1
            shard_length.append(length)
    split_info = [
        tfds.core.SplitInfo(
            name="train",
            shard_lengths=shard_length,
            num_bytes=0,
            filename_template=sharded_template_generator,
        )
    ]
    tfds.folder_dataset.write_metadata(
        data_dir=path.as_posix(),
        features=FEATURE_DICTIONARY,
        filename_template="{DATASET}-{SPLIT}-{SHARD_X_OF_Y}.{FILEFORMAT}",
        split_infos=split_info,
    )
    return shard_length

In [17]:
# Shards and writes the recipes5k_dataset into 10 shards and saves them in the specified directory.
shard_length = shard_and_write(
    recipes5k_dataset,
    10,
    f"D:\\food_v5\\Food Datasets\\final-dataset\\tfrecord\\{RECIPES5K.name}\\1.0.0",
    RECIPES5K.name,
)

Metadata written. Testing by reading first example. Set check_data=False to skip.


In [18]:
# Shards and writes the food101_dataset into 30 shards and saves them in the specified directory.
shard_length2 = shard_and_write(
    food101_dataset,
    30,
    f"D:\\food_v5\\Food Datasets\\final-dataset\\tfrecord\\{FOOD101.name}\\1.1.0",
    FOOD101.name,
)

Metadata written. Testing by reading first example. Set check_data=False to skip.


In [19]:
# Shards and writes the nutrition5k_dataset into 20 shards and saves them in the specified directory.
shard_length3 = shard_and_write(
    nutrition5k_dataset,
    20,
    f"D:\\food_v5\\Food Datasets\\final-dataset\\tfrecord\\{NUTRITION5K.name}\\1.0.0",
    NUTRITION5K.name,
)

Metadata written. Testing by reading first example. Set check_data=False to skip.


# Exported 

Creates a SimpleNamespace called EXPORTED containing the datasets and the one-hot encoder.

Exports the encoded categories and ingredients to JSON files for later use.

In [20]:
EXPORTED = {"datasets": DATASETS, "one_hot_encoder": ONE_HOT_ENCODER}

In [21]:
EXPORTED = SimpleNamespace(**EXPORTED)

## Export Encoded Categories and Ingredients for Decoding
Saves the encoded food categories and ingredients to JSON files for decoding during model inference.

In [ ]:
json.dump(
    ONE_HOT_ENCODER.all_food_categories_integer_encoded,
    open("./encoded_food_categories.json", "w"),
)

In [23]:
json.dump(
    ONE_HOT_ENCODER.all_ingredients_integer_encoded,
    open("./encoded_ingredients.json", "w"),
)

# Data Statistics

## Categories Statistics

In [24]:
print(f"The total number of ingredients : {len(ONE_HOT_ENCODER.all_food_categories)}")

The total number of ingredients : 102


In [25]:
df = pd.concat([RECIPES5K.metadata, FOOD101.metadata])

In [26]:
df_grouped_categories = df.groupby("Category")

In [27]:
print(
    f"The lowest number of images per category : {df_grouped_categories.size().min()}"
)
print(
    f"The highest number of images per category : {df_grouped_categories.size().max()}"
)
total_imgs = df_grouped_categories.size().sum()
total_category = len(df_grouped_categories)

print(f"The average number of images per category : {total_imgs//total_category}")

The lowest number of images per category : 1008
The highest number of images per category : 1050
The average number of images per category : 1047


## Ingredients Statistics

In [28]:
print(f"The total number of ingredients : {len(ONE_HOT_ENCODER.all_ingredients)}")

The total number of ingredients : 1037


In [29]:
average = 0
for ing in df["Ingredients"]:
    count_ing = len(ing.split(","))
    average += count_ing
average = average // len(df)
print(
    f"The average number of ingredients for each dish (recipes5k + food101) : {average}"
)

The average number of ingredients for each dish (recipes5k + food101) : 9


In [30]:
df_nutrition5k = NUTRITION5K.metadata

In [31]:
average = 0
for ing in df_nutrition5k["Ingredients"]:
    count_ing = len(ing.split(","))
    average += count_ing
average = average // len(df_nutrition5k)
print(f"The average number of ingredients for each dish (nutrition5k) : {average}")

The average number of ingredients for each dish (nutrition5k) : 5


# Testing

In [32]:
RECIPES5K.get_tensors(0)

(<tf.Tensor: shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x00\xe0\x00\xe0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87

In [33]:
get_file_data(1, 2)

[b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x00\xe0\x00\xe0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x